# Run subset by time operation and concat

**Rooki** calls climate data operations on the **rook** processing service.

In [1]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import rooki
from rooki import operators as ops

**run subset**

In [2]:
wf = ops.Subset(
        ops.Input(
                'tas', [
                    'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r1i1p1f2.Amon.tas.gn.v20200417',
                    'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r2i1p1f2.Amon.tas.gn.v20200417',
                ]
        ),
        time="2000/2000",
        apply_fixes=True,
)

resp = wf.orchestrate()
resp.ok

True

**URLs in metalink document ...**

In [3]:
resp.download_urls()

['http://rook4.cloud.dkrz.de:80/outputs/rook/cb709798-9244-11ed-9058-fa163ed6c06f/tas_Amon_HadGEM3-GC31-MM_dcppA-hindcast_r1i1p1f2_gn_20000116-20001216.nc',
 'http://rook4.cloud.dkrz.de:80/outputs/rook/cb70ac1a-9244-11ed-9058-fa163ed6c06f/tas_Amon_HadGEM3-GC31-MM_dcppA-hindcast_r2i1p1f2_gn_20000116-20001216.nc']

**... and open with xarray**

In [4]:
dsets = resp.datasets()

In [5]:
ds = dsets[0]
ds

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, lat: 324, lon: 432)
Coordinates:
  * time       (time) object 2000-01-16 00:00:00 ... 2000-12-16 00:00:00
  * lat        (lat) float64 -89.72 -89.17 -88.61 -88.06 ... 88.61 89.17 89.72
  * lon        (lon) float64 0.4167 1.25 2.083 2.917 ... 357.1 357.9 358.7 359.6
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/43)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            DCPP
    branch_method:          no parent
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-05-26T09:34:32Z
    ...                     ...
    tracking_id:            hdl:21.14100/48a83cad-1597-4df4-9404-afc3ba9cd23f
    variable_id:            tas
    variable_name:          tas
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by Met Office Hadley Ce...
    cmor_version:           3.4.0

In [6]:
# ds.attrs

**concat result**

In [7]:
import xarray as xr

In [8]:
processed_ds = xr.concat(dsets, dim="realization").mean(
    dim="realization", skipna=True, keep_attrs=True)
#processed_ds = xr.concat(dsets, dim="realization")
processed_ds

<xarray.Dataset>
Dimensions:   (time: 12, lat: 324, bnds: 2, lon: 432)
Coordinates:
  * time      (time) object 2000-01-16 00:00:00 ... 2000-12-16 00:00:00
  * lat       (lat) float64 -89.72 -89.17 -88.61 -88.06 ... 88.61 89.17 89.72
  * lon       (lon) float64 0.4167 1.25 2.083 2.917 ... 357.1 357.9 358.7 359.6
    height    float64 1.5
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (lat, bnds) float64 -90.0 -89.44 -89.44 ... 89.44 89.44 90.0
    lon_bnds  (lon, bnds) float64 -1.018e-05 0.8333 0.8333 ... 359.2 359.2 360.0
    tas       (time, lat, lon) float32 251.8 251.8 251.8 ... 247.0 247.0 247.0
Attributes: (12/43)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            DCPP
    branch_method:          no parent
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-05-26T09:34:32Z
    ...                     ...
    tracking_id:            hdl:21.14100/48a83cad-1597-4df4-9404-afc3ba9cd23f
    variable_id:            tas
    variable_name:          tas
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by Met Office Hadley Ce...
    cmor_version:           3.4.0